In [1]:
%load_ext blackcellmagic

In [2]:
import os
os.chdir("..")

In [3]:
import pandas as pd
import numpy as np
import pickle 
import warnings

warnings.filterwarnings("ignore")

from src.data.process_pdfs import convert_pdf_to_txt

In [4]:
pd.set_option("display.max_rows", 5)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

## 1. Extract text from pdf (replace by more advanced technique)
- Parallelization: Split df in 8 parts, run in different Jupyter Notebooks simultaneously, store to pickle and recombine

In [5]:
df = pd.read_excel('data/interim/reports_labeled_clean.xlsx')
df

,ID,Identifier,Company_Name,ISIN,Ticker,Country_of_Exchange,Financial_Period_Absolute,Financial_Period_Relative,CSR_URL,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,SDG_6,SDG_7,SDG_8,SDG_9,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,CSR_Filename
0,0,888.L,888 Holdings PLC,GI000A0F6407,888,United Kingdom,2020,FY0,https://corporate.888.com/wp-content/uploads/2021/04/2020-Annual-Report.pdf,False,False,True,True,True,True,True,True,False,NaN,False,True,True,NaN,False,True,False,0_888.L_2020.pdf
1,1,A.N,Agilent Technologies Inc,US00846U1016,A,United States of America,2020,FY0,https://www.agilent.com/about/companyinfo/sustainability/Agilent-Report-CSR-2020.pdf,False,False,True,True,True,True,True,True,False,NaN,False,True,True,NaN,True,True,False,1_A.N_2020.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8154,12676,ZBH.N,Zimmer Biomet Holdings Inc,US98956P1021,ZBH,United States of America,2016,FY-4,https://www.responsibilityreports.com/HostedData/ResponsibilityReportArchive/z/NYSE_ZBH_2016.pdf,False,False,True,True,True,True,True,True,False,NaN,False,True,False,NaN,False,True,False,12676_ZBH.N_2016.pdf
8155,12677,ZBH.N,Zimmer Biomet Holdings Inc,US98956P1021,ZBH,United States of America,2015,FY-5,https://www.responsibilityreports.com/HostedData/ResponsibilityReportArchive/z/NYSE_ZBH_2015.pdf,False,False,True,True,False,True,True,False,False,NaN,False,True,False,NaN,False,False,False,12677_ZBH.N_2015.pdf


In [6]:
# TEMP
df1 = df[0:1000]
df2 = df[1000:2000]
df3 = df[2000:3000]
df4 = df[3000:4000]
df5 = df[4000:5000]
df6 = df[5000:6000]
df7 = df[6000:7000]
df8 = df[7000:]

In [ ]:
df2['CSR_Text'] = df2['CSR_Filename'].apply(convert_pdf_to_txt)

In [ ]:
df2.to_pickle('data/interim/df2.p')
#df.to_pickle('data/interim/reports_txt.p')

## 2. Clean text (replace by more advanced technique)

In [6]:
import re
import nltk

def preprocess_text(text):
    '''
    Preprocess a string.
    :parameter
        :param text: string - name of column containing text
    :return
        preprocessed text
    '''
    # Clean text
    # Remove linebreaks
    text = text.replace('\n',' ')
    # Remove all non-ascii characters
    text = text.encode("ascii", "ignore").decode('UTF-8')
    # Convert to lowercase, remove punctuations and characters and then strip
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    # Tokenize (convert from string to list)
    lst_text = text.split()
    
    # Remove Stopwords
    lst_text = [word for word in lst_text if word not in nltk.corpus.stopwords.words("english")]
                
    # Stemming (remove -ing, -ly, ...) - Not used because lemmatisation is used
    #ps = nltk.stem.porter.PorterStemmer()
    #lst_text = [ps.stem(word) for word in lst_text]
                
    # Lemmatisation (convert the word into root word)
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    lst_text = [lem.lemmatize(word) for word in lst_text]
            
    # Convert back to string from list
    text = " ".join(lst_text)
    
    return text

In [11]:
df['CSR_Text'] = df['CSR_Text'].apply(preprocess_text)
df

C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Identifier,Company_Name,ISIN,Ticker,Country_of_Exchange,Financial_Period_Absolute,Financial_Period_Relative,CSR_URL,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,SDG_6,SDG_7,SDG_8,SDG_9,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,CSR_Filename,CSR_Text
0,0,888.L,888 Holdings PLC,GI000A0F6407,888,United Kingdom,2020,FY0,https://corporate.888.com/wp-content/uploads/2021/04/2020-Annual-Report.pdf,False,False,True,True,True,True,True,True,False,NaN,False,True,True,NaN,False,True,False,0_888.L_2020.pdf,888 holding plc annual report account 2020 year strong growth 888 one world leading online betti...
1,1,A.N,Agilent Technologies Inc,US00846U1016,A,United States of America,2020,FY0,https://www.agilent.com/about/companyinfo/sustainability/Agilent-Report-CSR-2020.pdf,False,False,True,True,True,True,True,True,False,NaN,False,True,True,NaN,True,True,False,1_A.N_2020.pdf,delivering promise 2020 corporate social responsibility report 1 letter president stakeholder en...
2,2,AA.N,Alcoa Corp,US0138721065,AA,United States of America,2020,FY0,https://www.alcoa.com/sustainability/en/pdf/2020-Sustainability-Report.pdf,False,False,True,True,True,True,True,True,False,NaN,False,True,True,NaN,True,True,False,2_AA.N_2020.pdf,sustainability report2020 alcoa sustainability performance 2020 0921 ratio new active mining dis...
3,3,AAC.AX,Australian Agricultural Company Ltd,AU000000AAC9,AAC,Australia,2021,FY0,https://aaco.com.au/wp-content/uploads/2021/07/FY21-Sustainability-Update.pdf,False,False,True,True,False,False,True,False,False,NaN,False,False,True,NaN,False,False,False,3_AAC.AX_2021.pdf,fy 2 1 sustainability update avon down station update fy21 progress climate nature reduction ani...
4,4,AAF.L,Airtel Africa PLC,GB00BKDRYJ47,AAF,United Kingdom,2020,FY0,https://airtel.africa/assets/pdf/Airtel_Africa_AR2020.pdf,True,False,True,True,True,True,True,True,False,NaN,True,True,True,NaN,True,True,True,4_AAF.L_2020.pdf,airtel africa plc 5354 grosvenor street london w1k 3hu england r e l f r c p l c n n u l r e p r...


## 3. Classification

In [5]:
# TEMP
df = pd.read_excel('data/raw/un_data.xlsx')
df

,Text,Source,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,SDG_6,SDG_7,SDG_8,SDG_9,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17
0,SDG 1,UN Descriptions,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,No poverty,UN Descriptions,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1548,ODA for data and statistics increased from $591 million in 2015 to $693 million in 2018. Almost ...,Progress towards the Sustainable Development Goals : report of the Secretary,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1549,"For the period 2015–2019, 146 countries and territories have birth registration data that are at...",Progress towards the Sustainable Development Goals : report of the Secretary,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [7]:
# Clean and remove stopwords
df['Text'] = df['Text'].apply(preprocess_text)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

Y = df[['SDG_1', 'SDG_2', 'SDG_3', 'SDG_4', 'SDG_5', 'SDG_6', 'SDG_7', 'SDG_8', 'SDG_9', 'SDG_10', 'SDG_11', 'SDG_12', 'SDG_13', 'SDG_14', 'SDG_15', 'SDG_16', 'SDG_17']].to_numpy()

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df.Text)

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

In [11]:
from sklearn.model_selection import train_test_split
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, Y, test_size=0.3, random_state=42)

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss

def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test_tfidf)))
    print("Hamming score: {}".format(hamming_score(y_pred, y_test_tfidf)))
    print("---")    

In [15]:
nb_clf = MultinomialNB()
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=6, tol=None)
lr = LogisticRegression()

for classifier in [nb_clf, sgd, lr]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(x_train_tfidf, y_train_tfidf)
    y_pred = clf.predict(x_test_tfidf)
    print_score(y_pred, classifier)

Clf:  MultinomialNB
Hamming loss: 0.058570524984187224
Hamming score: 0.004301075268817204
---
Clf:  SGDClassifier
Hamming loss: 0.03137254901960784
Hamming score: 0.4956989247311828
---
Clf:  LogisticRegression
Hamming loss: 0.05578747628083491
Hamming score: 0.05161290322580645
---


In [35]:
clf = OneVsRestClassifier(lr)
clf.fit(x_train_tfidf, y_train_tfidf)
y_pred = clf.predict(x_test_tfidf)

In [36]:
from sklearn.metrics import accuracy_score
for i in range(17):
    print('SDG ' + str(i+1) + ': ' + str(accuracy_score(y_test_tfidf[:,i], y_pred[:,i])))
print('Total: ' + str(accuracy_score(y_test_tfidf, y_pred)))

SDG 1: 0.9655913978494624
SDG 2: 0.9354838709677419
SDG 3: 0.9139784946236559
SDG 4: 0.9505376344086022
SDG 5: 0.9548387096774194
SDG 6: 0.9376344086021505
SDG 7: 0.978494623655914
SDG 8: 0.9483870967741935
SDG 9: 0.9419354838709677
SDG 10: 0.9397849462365592
SDG 11: 0.9483870967741935
SDG 12: 0.9376344086021505
SDG 13: 0.9612903225806452
SDG 14: 0.9526881720430107
SDG 15: 0.9505376344086022
SDG 16: 0.9268817204301075
SDG 17: 0.9075268817204301
Total: 0.05161290322580645


In [37]:
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing

In [38]:
df_data = df['Text']
df_target = df['SDG_1'] 

In [39]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
import re, string

stemmer = PorterStemmer()
token_pattern = re.compile(r"(?u)\b\w\w+\b")
my_stopwords = set(stopwords.words('english'))

def tokenize(text):
    stems = []
    tokens = token_pattern.findall(text)
    for item in tokens:
        if item not in my_stopwords:
            stems.append(stemmer.stem(item))
    return stems

def generate_word_list(X, Y, feature_names, target_names):
    d = pd.DataFrame(X.toarray(), columns=feature_names)
    doc = d[ d>0 ].count()
    d = d.assign(target=Y)
    d = d.groupby(by='target').sum()
    d = d.transpose()
    d.columns = target_names
    total = d.sum(axis=1)
    d = d.assign(total_occurrences=total)
    d = d.assign(document_occurrences=doc)
    d = d.sort_values(by='total_occurrences', ascending=False)
    return d

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

count_vectorizer = CountVectorizer(tokenizer=tokenize)
count_matrix = count_vectorizer.fit_transform(df_data)

word_list = generate_word_list(count_matrix, df_target, count_vectorizer.get_feature_names(), df_target.unique())
display(word_list)

,1,0,total_occurrences,document_occurrences
per,1430,79,1509,629
cent,1209,68,1277,559
...,...,...,...,...
miner,1,0,1,1
000,1,0,1,1


In [41]:
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

pipeline = Pipeline([ ('vectorizer', TfidfVectorizer(tokenizer=tokenize)), ('estimator', MultinomialNB()) ])
accuracy = cross_val_score(pipeline, df_data, df_target, cv=cv, n_jobs=-1)
print("Baseline cross-validated accuracy: {}".format(accuracy.mean()))

Baseline cross-validated accuracy: 0.9541943432344224


In [42]:
from sklearn.tree import DecisionTreeClassifier

nested_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

pipeline = Pipeline([ ('vectorizer', None), ('estimator', None) ])
params = {
    'vectorizer': [CountVectorizer(tokenizer=tokenize, binary=True), CountVectorizer(tokenizer=tokenize), TfidfVectorizer(tokenizer=tokenize)],
    'vectorizer__min_df': [1, 10],
    'vectorizer__max_df': [0.5, 0.8, 1.0],
    'estimator': [MultinomialNB(), DecisionTreeClassifier()]
}

search = GridSearchCV(pipeline, params, cv=nested_cv, scoring='accuracy')

accuracy = cross_val_score(search, df_data, df_target, cv=cv, n_jobs=-1)
print("Optimal cross-validated accuracy: {}".format(accuracy.mean()))

Optimal cross-validated accuracy: 0.963870521144148
